Neighborhoods in Toronto

Importing important libraries for data wrangling and analysis

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
!wget -q -O tornato_data.html https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


In [3]:
df=pd.read_html('tornato_data.html')

In [4]:
dftable = df[0]
dftable.head(20)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


Checking Not assigned neighbourhood

In [20]:
for i,col in dftable.iterrows():
    if col.Neighborhood =='Not assigned':
        print(col.Neighborhood,col.Borough);

The Beaches East Toronto


In [21]:
dftable.shape

(103, 3)

In [18]:
dftable.Neighborhood.value_counts()

Downsview                                                                                                                                        4
Willowdale                                                                                                                                       2
Don Mills                                                                                                                                        2
Thorncliffe Park                                                                                                                                 1
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park                                                                            1
Forest Hill North & West                                                                                                                         1
Milliken / Agincourt North / Steeles East / L'Amoreaux East                                                           

Dropping records which has Borough as Not assigned

In [5]:
dftable.drop(dftable[dftable['Borough'] == 'Not assigned'].index, inplace = True) 

In [22]:
dftable.Borough.value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

Checking for duplicate Postal Code. Surprisingly we don't have any repetition.

In [27]:
dftable['Postal code'].value_counts()

M4E    1
M1N    1
M4M    1
M9R    1
M2H    1
M5A    1
M2K    1
M9P    1
M3J    1
M8Z    1
M1P    1
M1M    1
M5M    1
M9V    1
M7R    1
M4T    1
M1E    1
M1L    1
M9W    1
M5B    1
M4R    1
M5H    1
M3K    1
M9B    1
M4W    1
M4B    1
M1S    1
M1J    1
M2N    1
M4X    1
M6J    1
M1T    1
M9C    1
M5W    1
M6E    1
M6H    1
M1V    1
M8X    1
M7Y    1
M1W    1
M4L    1
M4C    1
M6L    1
M5J    1
M4S    1
M5P    1
M3A    1
M5C    1
M4G    1
M6C    1
M6R    1
M2R    1
M9N    1
M8V    1
M9A    1
M9L    1
M6S    1
M4H    1
M1K    1
M1X    1
M6M    1
M4P    1
M4J    1
M6N    1
M3N    1
M7A    1
M6K    1
M3L    1
M5G    1
M5X    1
M6A    1
M2P    1
M8W    1
M5V    1
M3B    1
M1B    1
M4N    1
M6G    1
M6B    1
M2L    1
M5N    1
M5R    1
M1R    1
M1G    1
M1H    1
M3H    1
M2M    1
M5T    1
M2J    1
M3C    1
M4V    1
M3M    1
M6P    1
M4K    1
M5S    1
M1C    1
M4A    1
M5K    1
M9M    1
M4Y    1
M5L    1
M5E    1
M8Y    1
Name: Postal code, dtype: int64

In [6]:
dftable.shape

(103, 3)

Fetching coordinates for each postal address using geocoder api.

In [33]:
!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\akhil\Anaconda3

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_1
  ratelim            conda-forge/noarch::ratelim-0.1.6-py_2




geocoder-1.38.1      | 53 KB     |            |   0% 
geocoder-1.38.1      | 53 KB     | ###        |  30% 
geocoder-1.38.1      | 53 KB     | ########## | 100% 

ratelim-0.1.6        | 6 KB      |            |   0% 
ratelim-0.1.6        | 6 K

In [ ]:
# initialize your variable to None
# lat_lng_coords = None
# postal_code = 'M3A'
# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]
# Print('Coordinates',latitude,longitude)

In [15]:
dfPostalCode = pd.read_csv('Geospatial_Coordinates.csv')
dfPostalCode.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
dftable.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [19]:
Latitude= []
Longitude =[]
def AddCordinates(row):
    for i,pRow in dfPostalCode.iterrows():
        if pRow['Postal Code'] == row['Postal code']:
            Latitude.append(pRow['Latitude']);        
            Longitude.append(pRow['Longitude']);
            break;
        #     print(row["Postal code"]);
dftable.apply(AddCordinates,axis=1)

2      None
3      None
4      None
5      None
6      None
8      None
9      None
11     None
12     None
13     None
14     None
17     None
18     None
20     None
21     None
22     None
23     None
26     None
27     None
30     None
31     None
32     None
36     None
39     None
40     None
41     None
45     None
46     None
47     None
48     None
49     None
50     None
54     None
55     None
56     None
57     None
58     None
59     None
63     None
64     None
65     None
66     None
67     None
68     None
72     None
73     None
74     None
75     None
76     None
77     None
80     None
81     None
82     None
83     None
84     None
85     None
86     None
89     None
90     None
91     None
92     None
93     None
94     None
95     None
98     None
99     None
100    None
102    None
103    None
104    None
107    None
108    None
109    None
111    None
112    None
113    None
114    None
116    None
117    None
120    None
121    None
122    None
126    None
129 

In [18]:
dftable['Latitude'] = Latitude
dftable['Longitude'] = Longitude

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [23]:
dftable.head(23)

,Postal code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
5,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
8,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
9,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
11,M3B,North York,Don Mills,43.745906,-79.352188
12,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
